# Interacting with Assets in Markets API

This notebook provides a comprehensive guide on how to interact with different types of assets using the VAM API.
We will cover:
- Querying different asset types
- Filtering assets by execution venue
- Working with different asset types like spot, futures, and equity
- Performing batch operations



We will demonstrate how to query assets across these execution venues.


In [1]:
import os
from pathlib import Path

# Save the original working directory (only once)
try:
    original_wd
except NameError:
    original_wd = os.getcwd()

# Compute the target directory: two levels up from the original working directory
# For example, if original_wd is /Users/username/project/notebooks,
# then target_dir becomes /Users/username
target_dir = Path(original_wd).parents[2]

# Change the working directory to the target directory
os.chdir(target_dir)
print("Working directory set to:", os.getcwd())

Working directory set to: /home/jose/code/MainSequenceClientSide/mainsequence-sdk


In [3]:

import dotenv
env_loaded=dotenv.load_dotenv('.env')
print(f"Env file loaded: {env_loaded}")
import mainsequence 
import mainsequence.client as msc

# Define execution venue symbols
BINANCE_EV = msc.MARKETS_CONSTANTS.BINANCE_EV_SYMBOL
BINANCE_FUTURES_EV = msc.MARKETS_CONSTANTS.BINANCE_FUTURES_EV_SYMBOL
ALPACA_EV = msc.MARKETS_CONSTANTS.ALPACA_EV_SYMBOL

Env file loaded: True


2025-08-07T11:11:30.863770Z [debug    ] Getting Auth Headers ASSETS_ORM application_name=ms-sdk command_id=None data_source_id=2 job_run_id=None project_id=1 (at utils.py:51 in refresh_headers())
2025-08-07T11:11:31.141581Z [debug    ] Getting Auth Headers ASSETS_ORM application_name=ms-sdk command_id=None data_source_id=2 job_run_id=None project_id=1 (at utils.py:51 in refresh_headers())
2025-08-07T11:11:31.524003Z [debug    ] took 0.3696 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/pods/projects/get_user_default_project/ application_name=ms-sdk command_id=None data_source_id=2 job_run_id=None project_id=1 (at utils.py:107 in make_request())
2025-08-07T11:11:31.524901Z [info     ] Set remote data source to orm_class='DataSource' id=2 display_name='Default DB' organization=2 class_type='timescale_db' status='AVAILABLE' extra_arguments=None application_name=ms-sdk command_id=None data_source_id=2 job_run_id=None project_id=1 (at models_tdag.py:1998 in set_remote_db())
2025

## Optimal Asset Queries

There are many assets, and there will always be more. We recommend narrowing your search so that the expected results are in the hundreds. If you need more assets, it is best to first create a category that encompasses those assets and then query by that category.

A good way to start narrowing asset searches is by filtering on the execution venue and some FIGI properties. For example, if you want to retrieve the equivalent assets from one category on another exchange, you could do something like this:



In [7]:
top_100_cryptos = msc.AssetCategory.get(unique_identifier="top_100_crypto_market_cap")



quote_asset=msc.Asset.get(ticker='USDT',
                              security_market_sector=msc.MARKETS_CONSTANTS.FIGI_MARKET_SECTOR_CURNCY,
                              security_type=msc.MARKETS_CONSTANTS.FIGI_SECURITY_TYPE_CRYPTO
                              )

# Get them through the main sequence FIGI class and exchange
binance_currency_pairs = msc.AssetCurrencyPair.filter(
    base_asset__id__in=top_100_cryptos.assets,
    quote_asset__id=quote_asset.id,
    include_base_quote_detail=False
)



2025-08-07T11:18:25.982695Z [debug    ] took 0.4541 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/assets/asset-category/ application_name=ms-sdk command_id=None data_source_id=2 job_run_id=None project_id=1 (at utils.py:107 in make_request())
2025-08-07T11:18:26.452285Z [debug    ] took 0.4686 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/assets/asset/ application_name=ms-sdk command_id=None data_source_id=2 job_run_id=None project_id=1 (at utils.py:107 in make_request())
2025-08-07T11:18:27.079046Z [debug    ] took 0.6257 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/assets/asset_currency_pair/ application_name=ms-sdk command_id=None data_source_id=2 job_run_id=None project_id=1 (at utils.py:107 in make_request())


## Filtering Assets by Type

Different asset types have distinct properties, which can help you extend your filters. It's important to note that all asset types inherit from the Asset class. Therefore, you can always use the Asset class to access all assets and their general properties.

In [11]:
# Filtering Simple Assets
simple_assets = msc.Asset.filter(ticker="BTCUSDT")
print(f" {len(simple_assets)}")

# Filtering Futures
cash_equity_assets = msc.AssetCurrencyPair.filter(ticker="BTCUSDT")
print(f"Total Assets CurrencyPair Assets : {len(cash_equity_assets)}")

# Filtering  Futures USDM Assets
futures_assets= msc.AssetFutureUSDM.filter(ticker="BTCUSDT")
print(f"Total Futures Assets : {len(futures_assets)}")




2025-08-07T11:18:44.838120Z [debug    ] took 0.3930 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/assets/asset/ application_name=ms-sdk command_id=None data_source_id=2 job_run_id=None project_id=1 (at utils.py:107 in make_request())


Total Spot Crypto Assets on Binance: 0


2025-08-07T11:18:45.235828Z [debug    ] took 0.3966 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/assets/asset_currency_pair/ application_name=ms-sdk command_id=None data_source_id=2 job_run_id=None project_id=1 (at utils.py:107 in make_request())


Total Assets CurrencyPair Assets : 0


2025-08-07T11:18:45.638785Z [debug    ] took 0.4018 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/assets/asset_future_usdm/ application_name=ms-sdk command_id=None data_source_id=2 job_run_id=None project_id=1 (at utils.py:107 in make_request())


Total Futures Assets : 0


## Using `filter_with_asset_class`

The `filter_with_asset_class` method is an extended version of the `filter` method.
It ensures that each returned asset is an instance of its correct class 

### When to Use?
- When querying multiple asset types and needing them returned with the correct class.
- When working with assets across execution venues and requiring proper type differentiation.

### Example 1: Query all asset types for a symbol


In [12]:
# Query Binance Spot assets with correct asset classes
all_btc_assets = msc.Asset.filter_with_asset_class(ticker="BTCUSDT")

# Displaying asset information
for asset in all_btc_assets:
    print(f"Asset: {asset.ticker},  Class: {type(asset).__name__}")


2025-08-07T11:19:47.503654Z [debug    ] took 0.3935 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/assets/asset/list_with_asset_class/ application_name=ms-sdk command_id=None data_source_id=2 job_run_id=None project_id=1 (at utils.py:107 in make_request())


### Example 2: Using FIGI Information to Narrow Down Results

Now imagine we only want currency pairs instead of futures.
 In this case, we can use FIGI details to identify the asset classes we are specifically interested in.



In [13]:
all_curreny_pairs_bitcoin= msc.Asset.filter_with_asset_class(ticker="BTCUSDT",
security_market_sector=msc.MARKETS_CONSTANTS.FIGI_MARKET_SECTOR_CURNCY,
security_type=msc.MARKETS_CONSTANTS.FIGI_SECURITY_TYPE_CRYPTO
)

print(all_curreny_pairs_bitcoin)

2025-08-07T11:20:13.501934Z [debug    ] took 0.3968 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/assets/asset/list_with_asset_class/ application_name=ms-sdk command_id=None data_source_id=2 job_run_id=None project_id=1 (at utils.py:107 in make_request())


[]
